In [ ]:
!pip install instructor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.6/345.6 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: jiter
    Found existing installation: jiter 0.9.0
    Uninstalling jiter-0.9.0:
      Successfully uninstalled jiter-0.9.0


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.7 MB/s eta 0:00:00


In [ ]:
import os
import instructor
from pydantic import BaseModel
from openai import OpenAI
import getpass

In [ ]:
#API_KEY = getpass.getpass("Enter your OpenAI API key: ")
API_KEY  = # Insert OpenAI Key here
open_ai = OpenAI(api_key=API_KEY)
client  = instructor.from_openai(open_ai, mode=instructor.Mode.TOOLS_STRICT)

In [ ]:
class ZeroShotResponse(BaseModel):
    filler: str
    message: str
def ZeroShotPrompting(message):
    return client.chat.completions.create(
        model="gpt-4o-mini",
        response_model=ZeroShotResponse,
        messages=[
            {
                "role": "system",
                "content": """You have excellent knowledge of interactive theorem prover Isabelle .
                              You will try your best to solve the query given by the user.""",
            },
            {
                "role": "user",
                "content": f"""You are given a mathematical statement in natural lanaguage written in LaTex, i.e Natural Language:"{message}". Translate the natural language version to an Isabelle version. You only need to translate the statement into Isabelle not solve it.
                Remember the output that you give is structured as filler and message. The message part should strictly contain only Isabelle translation and nothing else. Everthing else should be in filler. Transalation should only be in Isabelle version.
                """,
            },
        ],
    )

In [ ]:
class FewShotResponse(BaseModel):
    filler: str
    message: str
def FewShotPrompting(message,examples):
  return client.chat.completions.create(
      model="gpt-4o-mini",
      response_model=FewShotResponse,
      messages=[
          {
              "role": "system",
              "content": """You have excellent knowledge of interactive theorem prover Isabelle .
                            You will try your best to solve the query given by the user.""",
          },
          {
               "role": "user",
                "content": f"""You are given a mathematical statement in natural lanaguage written in LaTex, i.e Natural Language:"{message}".
                Translate the natural language version to an Isabelle version. You only need to translate the statement into Isabelle not solve it.
                For reference, I will give few examples from algebra and number theory. In each example has <Natural language version> tag which encloses latex version of statement and
                <Isabelle Version> tag which encloses the Isabelle translation of the statement. The examples are as follows:
                Algebra Examples:
                {examples[0]}
                Number Theory Examples:
                {examples[1]}
                Remember the output that you give is structured as filler and message. The message part should strictly contain only Isabelle translation and nothing else.
                Everthing else should be in filler. Transalation should only be in Isabelle version and in prediction dont include <Natural language version> or <Isabelle Version> tags.
                """,
            },
        ],
    )

In [ ]:
import json

In [ ]:
with open("valid_data.json","r") as f:
  data=json.load(f)


In [ ]:
class Auto():
  def __init__(self, prompting):
    self.prompting = prompting

  def response(self,dataset):
    y_pred=[]
    y_test=[]
    for message in dataset:
      print(message)
      res=self.prompting(dataset[message]["natural_s"])
      y_pred.append(res.message)
      y_test.append(dataset[message]["formal_s"])
    return y_pred,y_test

  def evaluate(self,y_pred,y_test):
    import evaluate
    metric = evaluate.load("bleu")
    return metric.compute(predictions=y_pred, references=y_test)



In [ ]:
agent=Auto(ZeroShotPrompting)

In [ ]:
pred,test=agent.response(data)

In [ ]:
agent.evaluate(pred,test)

{'bleu': 0.293757982810891,
 'precisions': [0.5538213526782443,
  0.34268447362530774,
  0.24120373888593358,
  0.1843592721641502],
 'brevity_penalty': 0.9691955049289096,
 'length_ratio': 0.9696603666335086,
 'translation_length': 13647,
 'reference_length': 14074}

In [ ]:
count=20

for i in range(len(pred)):
  print("Truth")
  print(test[i])
  print("pred")
  print(pred[i])
  count-=0
  if count==0:
    break


Truth
theorem:
  shows "ceiling (sqrt 27) - floor (sqrt 26) = 1"
  
pred
import Data.Real.

lemma evaluate_sqrt : \<lceil\<sqrt>27\<rceil> - \<lfloor\<sqrt>26\<rfloor> = 1 \<rbrakk>
Truth
theorem:
  fixes a :: "nat \<Rightarrow> rat"
  assumes "a 1 = 1"
    and "a 2 = 3 / 7"
    and "\<forall> n. a (n + 2) = (a n * a (n + 1)) / (2 * a n - a (n + 1))" 
  shows "fst (quotient_of (a 2019)) + snd (quotient_of (a 2019)) = 8078"
  
pred
theory Recursive_Sequence
  imports Main
begin

(* Define the recursive sequence *)
fun a :: "nat \<Rightarrow> rational" where
  "a 1 = 1"
| "a 2 = 3 / 7"
| "a n = (a (n - 2) * a (n - 1)) / (2 * a (n - 2) - a (n - 1)) if n \ge> 3" 


(* Statement to prove *)
value_2019_thm:
  shows "a 2019 = p / q" where "gcd p q = 1" and "p > 0" and "q > 0"
  by (auto)

end
Truth
theorem:
  fixes x :: real
  assumes h0 : "x^3 - (x+1) * (x-1) * x = 5"
  shows "x^3 = 125"

pred
locale cube_volume =
  fixes side_length :: nat
  assumes "side_length > 0"
  defines "volume_cube 

In [ ]:
f = open("algebra_prompt.txt", "r")
algebra=f.read()
f.close()

In [ ]:
f = open("number_theory_prompt.txt", "r")
number_theory=f.read()
f.close()

In [ ]:
examples=[algebra,number_theory]


In [ ]:
class MultiAuto(Auto):
  def response(self,dataset,examples):
    y_pred=[]
    y_test=[]
    for message in dataset:
      print(message)
      res=self.prompting(dataset[message]["natural_s"],examples)
      y_pred.append(res.message)
      y_test.append(dataset[message]["formal_s"])
    return y_pred,y_test

In [ ]:
agent=MultiAuto(FewShotPrompting)

In [ ]:
pred,test=agent.response(data,examples)

mathd_algebra_151
amc12a_2019_p9
amc12a_2009_p5
mathd_algebra_11
amc12a_2013_p8
mathd_algebra_192
amc12a_2016_p2
imo_1987_p4
imo_1962_p4
mathd_algebra_109
numbertheory_sqmod4in01d
mathd_algebra_245
amc12a_2019_p21
amc12b_2020_p5
imo_1984_p2
amc12b_2002_p6
amc12b_2003_p6
induction_sum2kp1npqsqm1
mathd_algebra_433
amc12_2000_p11
amc12a_2013_p7
mathd_numbertheory_709
mathd_algebra_131
mathd_numbertheory_198
mathd_numbertheory_530
mathd_numbertheory_169
mathd_numbertheory_640
mathd_algebra_55
numbertheory_2dvd4expn
amc12a_2002_p1
algebra_xmysqpymzsqpzmxsqeqxyz_xpypzp6dvdx3y3z3
amc12b_2002_p11
imo_1966_p5
amc12a_2008_p4
mathd_algebra_140
amc12a_2003_p1
numbertheory_sumkmulnckeqnmul2pownm1
mathd_algebra_616
imo_2006_p3
mathd_algebra_69
mathd_numbertheory_269
mathd_algebra_37
mathd_numbertheory_466
aime_1984_p5
mathd_numbertheory_81
mathd_numbertheory_236
algebra_manipexpr_2erprsqpesqeqnrpnesq
numbertheory_sqmod3in01d
algebra_2rootsintpoly_am10tap11eqasqpam110
aime_1997_p11
mathd_algebra_509


In [ ]:
agent.evaluate(pred,test)


{'bleu': 0.40248359675883394,
 'precisions': [0.7890493488083636,
  0.5634338494974191,
  0.4420779701824243,
  0.3585030791094268],
 'brevity_penalty': 0.7812022210785711,
 'length_ratio': 0.8019752735540714,
 'translation_length': 11287,
 'reference_length': 14074}

In [ ]:
count=20

for i in range(len(pred)):
  print("Truth")
  print(test[i])
  print("pred")
  print(pred[i])
  count-=0
  if count==0:
    break


Truth
theorem:
  shows "ceiling (sqrt 27) - floor (sqrt 26) = 1"
  
pred
theorem
	shows "ceil (sqrt(27)) - floor (sqrt(26)) = 1"
Truth
theorem:
  fixes a :: "nat \<Rightarrow> rat"
  assumes "a 1 = 1"
    and "a 2 = 3 / 7"
    and "\<forall> n. a (n + 2) = (a n * a (n + 1)) / (2 * a n - a (n + 1))" 
  shows "fst (quotient_of (a 2019)) + snd (quotient_of (a 2019)) = 8078"
  
pred
theorem
	fixes a :: "nat \Rightarrow rational"
	assumes "a 1 = 1"
	and "a 2 = 3 / 7"
	and "a n = (a (n - 2) * a (n - 1)) / (2 * a (n - 2) - a (n - 1))"
	and "n \geq 3"
shows "a 2019 = p/q" for relatively prime positive integers p and q
Truth
theorem:
  fixes x :: real
  assumes h0 : "x^3 - (x+1) * (x-1) * x = 5"
  shows "x^3 = 125"

pred
theorem
	fixes a v :: real
	assumes "v = a^3"
	and "(a + 1) * (a - 1) * a = v - 5"
	shows "v = 125"
Truth
theorem:
  fixes a b :: real
  assumes h0 : "a \<noteq> b"
    and h1 : "a \<noteq> 2 * b"
    and h2 : "(4*a+3*b) / (a-2*b) = 5"
  shows "(a+11*b) / (a-b) = 2"

pred
theor